TODO: find how many corrections are of the form I - A - I' - A' and in which A-A' is at the threshold 

In [ ]:
map_relations = {'Comment':0, 'Contrast':1, 'Correction':2, 'Question-answer_pair':3, 'Acknowledgement':4,'Elaboration':5,
                 'Clarification_question':6, 'Conditional':7, 'Continuation':8, 'Result':9, 'Explanation':10, 'Q-Elab':11,
                 'Alternation':12, 'Narration':13, 'Confirmation_question':14, 'Sequence':15, 'Break':16}

In [2]:
reverse_relations = {0:'Comment', 1:'Contrast', 2:'Correction', 3:'Question-answer_pair', 4:'Acknowledgement',5:'Elaboration',
                 6:'Clarification_question', 7:'Conditional', 8:'Continuation', 9:'Result', 10:'Explanation', 11:'Q-Elab',
                 12:'Alternation', 13:'Narration', 14:'Confirmation_question', 15:'Sequence', 16:'Break'}

In [1]:
#import pickle
import json
#from collections import Counter, defaultdict 
from sklearn.metrics import precision_recall_fscore_support,classification_report, ConfusionMatrixDisplay, confusion_matrix

In [2]:
home = %pwd
annotations = '/home/kate/minecraft_utils/llm_annotator/annotated_data'
gold_path = annotations + '/DEV_32_bert.json'
# preds_path = home + '/pickles/scores_multi_d10.pkl'

In [3]:
with open(gold_path, 'r') as jf:
    dev = json.load(jf)

In [4]:
len(dev)

32

Keep only Correction relations

In [5]:
for game in dev:
    # rels = game['predicted_relations']
    rels = game['relations']
    new_rels = []

    #add incoming Result information
    for rel in rels:
        if rel['type'] == 'Correction':
            new_rels.append(rel)
    game['relations'] = new_rels

In [6]:
def is_nl(edu):
    """
    if every word in alphanumeric
    """
    nl = 1
    words = edu.split(' ')
    # print(words)
    for word in [w for w in words if w != '']:
        if not contains_number(word) or len(word)!=5:
            nl = 0
            break
    return nl

def contains_number(string):
    return any(char.isdigit() for char in string)

In [7]:
#measure of target undoing source
def undone(source, target):
    source_text = source.split()
    target_text = target.split()
    source_ones = []
    source_zeros = []
    target_ones = []
    target_zeros = []
    for s in source_text:
        if s[0] == '1':
            source_ones.append(s[1:])
        else:
            source_zeros.append(s[1:])
    for t in target_text:
        if t[0] == '1':
            target_ones.append(t[1:])
        else:
            target_zeros.append(t[1:])
    total = len(target_text) #try with target text as a denominator
    count = 0
    for e in source_ones: 
        if e in target_zeros:
            count += 1
    for e in source_zeros:
        if e in target_ones:
            count += 1
    #count is number of target moves that undo source moves
    return round(count/total, 2)

In [8]:
#STEP 1: find all moves in game
for game in dev:
# for game in [g for g in test_gold if g['id'] == 'C145-B35-A15']:
    # print(game['id'])
    edus = game['edus']
    corrections = []
    moves = [i for i,e in enumerate(game['edus']) if is_nl(e['text']) and len(e['text']) > 0]
    pairs = [(moves[r], moves[r+1]) for r in range(len(moves)-1)]
    for p in pairs:
        # print(edus[p[0]]['text'])
        # print(edus[p[1]]['text'])
        # print('------------')
        if undone(edus[p[0]]['text'], edus[p[1]]['text']) > 0.75:
            corrections.append(p)
    #STEP 2: find first architect ling move between them
    pred_corrections = []
    for co in corrections:
        for i, edu in enumerate(edus[co[0]:co[1]]):
            if edu['speaker'] == 'Architect': #but what if there is no arch between?
                pred_corrections.append(co)
                pred_corrections.append((co[0], co[0]+i))
    #STEP 3: add corrections to game
    pred_rels = []
    for rel in pred_corrections:
        new = {}
        new['x'] = rel[0]
        new['y'] = rel[1]
        new['type'] = 'Correction'
        pred_rels.append(new)
    game['pred_rels'] = pred_rels


Now compare gold to predicted Corrections

In [10]:
#now rearrange
gold_corrections = []
pred_corrections = []

for game in dev:
    preds = [(rel['x'], rel['y']) for rel in game['pred_rels']]
    gold = [(rel['x'], rel['y']) for rel in game['relations']]
    for r in gold:
        if r in preds:
            gold_corrections.append(1)
            pred_corrections.append(1)
        if r not in preds:
            gold_corrections.append(1)
            pred_corrections.append(0)
    for r in preds:
        if r not in gold:
            gold_corrections.append(0)
            pred_corrections.append(1)
            

In [11]:
len(gold_corrections), len(pred_corrections)

(186, 186)

In [12]:
scores = precision_recall_fscore_support(gold_corrections, pred_corrections, average='binary')

In [13]:
#using 50 threshold
scores

(0.2391304347826087, 0.1896551724137931, 0.21153846153846154, None)

In [21]:
#visualize 
correct = []
others = []

for game in test_gold[:1]:
    print('--------', game['id'], '--------')
    edus = game['edus']
    preds = [(rel['x'], rel['y']) for rel in game['pred_rels']]
    gold = [(rel['x'], rel['y']) for rel in game['relations']]
    print('\nCorrect:')
    for r in preds:
        if r in gold:
            snip = edus[r[0]:r[1]+1]
            for s in snip:
                print(s['text'])
            print('------------------------------------------------')
    print('\nIncorrect:')
    for r in preds:
        if r not in gold:
            snip = edus[r[0]:r[1]+1]
            for s in snip:
                print(s['text'])
            print('-------------------------------------------------')

                
            

-------- C54-B13-A30 --------

Correct:

 Incorrect
0ym3r 0ym2r 0ym2x 0ym1x 0ym1r 0ym1y 1yj1y 1yj1r 1yj1x 1yj2r 1yj2x 1yj3r 
yes exactly!
ok now delete the boxes underneath,
leaving only the steps
so far we've built an L,
a large duck,
a funky staircase,
and a cube shaped house.
How would you describe the end goal of this build?
abstract.
there are a lot of stair type pieces in red yellow and orange
cool,
that might help,
maybe...
haha
lol.
ok now delete the bottom pieces of the stairs
leaving only the parts you would use for steps
0yj2r 0yj1r 0yj1x 
---------------------------------
0ym3r 0ym2r 0ym2x 0ym1x 0ym1r 0ym1y 1yj1y 1yj1r 1yj1x 1yj2r 1yj2x 1yj3r 
yes exactly!
---------------------------------
0ym3r 0ym2r 0ym2x 0ym1x 0ym1r 0ym1y 1yj1y 1yj1r 1yj1x 1yj2r 1yj2x 1yj3r 
yes exactly!
ok now delete the boxes underneath,
leaving only the steps
so far we've built an L,
a large duck,
a funky staircase,
and a cube shaped house.
How would you describe the end goal of this build?
abstract.
